# Cours ENPC - Pratique du calcul scientifique

## Résolution de systèmes non linéaires avec introduction à la différentiation automatique

--------------

### Exercice sur Newton-Raphson en dimension 2

On considère le système non linéaire suivant
$$
\left \{
    \begin{aligned}
        &y = (x-1)^2 \\
        &x^2 + y^2 = 4
    \end{aligned}
\right.
$$

1. En exploitant des tracés graphiques appropriés pour visualiser grossièrement la (les) zone(s) contenant une (des) solution(s), implémenter un algorithme de Newton-Raphson pour calculer une approximation de cette (ces) solution(s) en choississant un (des) point(s) initial(aux) judicieux.

1. Modifier l'algorithme pour recueillir l'ensemble des itérations et visualiser l'évolution de l'erreur en fonction de l'itération pour diverses valeurs initiales en utilisant une échelle appropriée en y. On pourra estimer qu'une solution donnée est fournie par la valeur finale de l'algorithme.

--------------------
--------------------
--------------------

### Exercice sur la méthode babylonienne

Soit un paramètre réel $a>0$ et la suite définie par
<a id="baby"></a>
$$
\tag{1}
x_0>0 \qquad ; \qquad ∀k∈\mathbb{N},\quad x_{k+1}=\frac{1}{2}\left(x_k+\frac{a}{x_k}\right)
$$

1. En écrivant $x_{k+1}-\sqrt{a}$ en fonction de $x_k-\sqrt{a}$ puis $x_{k+1}-x_k$, montrer que $(x_k)$ converge quadratiquement vers $x_*=\sqrt{a}$ quel que soit $x_0>0$.

1. Montrer que la formulation par récurrence <a href="#baby">(1)</a> n'est autre que l'algorithme de Newton-Raphson appliqué à une fonction à identifier s'annulant en $x_*=\sqrt{a}$.

1. Construire une fonction `Babylonian` prenant comme arguments `a` ainsi qu'un entier `n` valant `10` par défaut et qui renvoie le vecteur $[x_0,x_1,\ldots,x_n]$ en initialisant la suite à $x_0=\frac{1+a}{2}$.

1. Tester sur quelques valeurs.

1. Tracer l'erreur $|x_k-x_*|$ en fonction du rang $k$ pour $a=2$.

   L'idée de la suite de l'exercice est d'appliquer la fonction `Babylonian` définie plus haut à un argument `a` non pas de type `Float64` mais d'un nouveau type permettant d'estimer à la fois la valeur de $\sqrt{a}$ mais également de la dérivée de $a\mapsto\sqrt{a}$ soit $\frac{1}{2\sqrt{a}}$. Pour cela, on introduit de nouveaux nombres qu'on appelle **nombres duaux**. Ceux-ci sont définis à l'instar des nombres complexes à partir de la définition d'un nombre particulier noté $ɛ$ de sorte qu'un nombre dual s'écrit $x=a+bɛ$ avec $a$ et $b$ réels. En quelque sorte $ɛ$ joue un rôle analogue au $i$ complexe à la différence que l'on pose ici $ɛ^2=0$. L'objectif de tels nombres est d'être en mesure de stocker à la fois la valeur d'une fonction mais également sa dérivée en posant

   <a id="fdual"></a>
   $$
   \tag{2}
   f(a+bɛ)=f(a)+f'(a)bɛ
   $$

   ce qui entraîne que la dérivée en $a$ de $f$ peut être obtenue en récupérant la composante sur $ɛ$ de $f(a+ɛ)$ (i.e. en prenant $b=1$).

   En pratique, il est donc nécessaire de redéfinir le comportement des fonctions usuelles en cohérence avec <a href="#fdual">(2)</a>. Toutefois dans l'application actuelle, seules les opérations `+`, `-`, `*` et `/` seront nécessaires et donc devront être surchargées pour pouvoir prendre comme arguments des nombres duaux. En outre, il sera également nécessaire d'implémenter les fonctions `convert` pour convertir un réel en nombre dual et `promote_rule` pour exprimer le fait qu'en présence d'une opération impliquant deux nombres dont l'un est dual, les deux doivent d'abord être exprimés sous forme de nombres duaux avant de lancer l'opération. À noter que la surcharge des opérateurs et fonctions de base n'est possible que si ceux-ci sont explicitement importés à l'aide par exemple de `import Base: +, -, ...`. Il est également possible de définir la fonction `Base.show` de manière à ce que l'affichage d'un nombre dual ait explicitement la forme `a+bɛ`.

   La surcharge des opérateurs s'exprime mathématiquement par
   $$
   \begin{align*}
   (a+bɛ)+(c+dɛ)&=(a+c)+(b+d)ɛ\\
   (a+bɛ)-(c+dɛ)&=(a-c)+(b-d)ɛ\\
   (a+bɛ)*(c+dɛ)&=ac+(bc+ad)ɛ\\
   (a+bɛ)/(c+dɛ)&=\frac{a}{c}+\frac{bc-ad}{c^2}ɛ
   \end{align*}
   $$

   Alternativement à cette dernière opération, on peut aussi définir $\mathrm{inv}(a+bɛ)=\frac{1}{a}-\frac{b}{a^2}ɛ$ puis $u/v=u*\mathrm{inv}(v)$.

1. Étudier le `struct D` défini pour représenter un nombre dual ainsi que les lignes de code associées. Compléter les parties de code manquantes.

1. Définir une instance du nombre `ɛ` (\varepsilon puis touche TAB pour afficher ε), en d'autres termes le nombre `0+1ɛ` et effectuer quelques opérations pour vérifier les implémentations, par exemple

   ```julia
   (1+2ɛ)*(3+4ɛ)
   1/(1+ɛ)
   (1+2ɛ)/(2-ɛ)
   ```

1. Exploiter la structure de nombre dual pour estimer la dérivée de la fonction racine à partir de la méthode babylonienne (en exploitant directement la fonction `Babylonian` sans la réécrire) sur quelques exemples ($a=0.1$, $a=2$, $a=25$) et vérifier les résultats avec la solution analytique.

1. Superposer sur un graphe la dérivée de la racine obtenue par la méthode babylonienne sur nombre dual et l'expression analytique.

1. Proposer une méthode analogue pour calculer la racine $p^\textrm{ème}$ d'un nombre $a$ i.e. $\sqrt[p]{a}$. Vérifier que la dérivée de la racine $p^\textrm{ème}$ peut également être obtenue par exploitation des nombres duaux sans ligne de code supplémentaire.

In [ ]:
function Babylonian(a; n = 10)
    # Votre code ici
end

In [ ]:
for a in (0.1, 2, 25) 
    # Votre code de vérification de l'algorithme ici
end

In [ ]:
using Plots

# Votre code de tracé ici des erreurs ici

In [ ]:
import Base: +, -, *, /, inv, conj, convert, promote_rule
using LinearAlgebra

struct D <: Number
    f::Tuple{Float64,Float64}
end
+(x::D, y::D) = D(x.f .+ y.f)
-(x::D, y::D) = # à compléter
*(x::D, y::D) = D((x.f[1]*y.f[1], (x.f[2]*y.f[1] + x.f[1]*y.f[2])))
/(x::D, y::D) = # à compléter
convert(::Type{D}, x::Real) = D((x,zero(x)))
promote_rule(::Type{D}, ::Type{<:Real}) = D
Base.show(io::IO,x::D) = print(io,x.f[1],x.f[2]<0 ? " - " : " + ",abs(x.f[2])," ε")
ε = D((0,1))

In [ ]:
@show (1+2ɛ)*(3+4ɛ) ; @assert (1+2ɛ)*(3+4ɛ) == 3+10ɛ "erreur"
# Compléter avec d'autres vérifications
;

In [ ]:
for a in (0.1, 2, 25) 
    # Votre code de vérification de l'algorithme avec nombres duaux ici
end

In [ ]:
# Votre code avec les tracés ici

In [ ]:
function nthrt(a, p=2; x=1, n=10)
   # Votre code sur l'extension à la racine pième ici 
end

In [ ]:
a = 2 ; p = 3
# Votre code pour comparer le résultat de `nthrt` à la solution analytique

--------------------
--------------------
--------------------

### Extension de la différentiation automatique au second ordre et à plusieurs variables : application à une méthode des moindres carrés non linéaire

L'objectif de cet exercice est d'étendre le concept de différentiation automatique au cas d'une fonction scalaire de plusieurs variables au second ordre, autrement dit de permettre le calcul du gradient et de la hessienne en $a$ de la fonction deux fois différentiable $f: Ω⊂\mathbb{R}^N → \mathbb{R}$.

En partant de son développement de Taylor à l'ordre 2

<a id="fvecdual"></a>
$$
\tag{3}
x₀, ɛ ∈ \mathbb{R}^N,\quad
f(x₀+ɛ)=f(x₀)+∇f(x₀)ᵀɛ+\frac{1}{2}ɛᵀ∇²f(x₀)ɛ+ℴ(ɛ²)
\quad\textrm{avec}\quad
f(x₀)∈\mathbb{R},\,∇f(x₀)∈\mathbb{R}^N,\,∇²f(x₀)∈\mathbb{R}^{N×N}
$$

on a l'idée d'introduire une nouvelle classe de nombres scalaires formés par un triplet constitué d'un scalaire $a$, d'un vecteur $b$ et d'une matrice carrée symétrique $c$ et d'écrire

<a id="vecdual"></a>
$$
\tag{4}
x=a+bᵀɛ+\frac{1}{2}\mathrm{Tr}(cη)
$$

où $ɛ$ et $η$ sont ici respectivement un vecteur de $\mathbb{R}^N$ et une matrice de $\mathbb{R}^{N×N}$ telles que $ɛɛᵀ=η$, $ηɛ=0$ et $η²=0$. On peut ainsi faire l'économie de $η$ dans <a href="#vecdual">(4)</a> et réécrire tout nombre $x$ sous la forme

<a id="vecdual2"></a>
$$
\tag{5}
x=a+bᵀɛ+\frac{1}{2}ɛᵀcɛ
$$

Une telle famille de nombres est représentée ci-dessous par un `struct DD{N} <: Number` (où `N` permet de paramétriser ce nouveau type par la dimension $N$ et `<: Number` indique que le type dérive du type abstrait `Number` donnant un sens aux opérations usuelles) contenant les données membres `val`, `grad` et `hess` désignant respectivement $a$, $b$ et $c$ dans la décomposition <a href="#vecdual2">(5)</a>. A noter que 3 constructeurs sont définis pour ce type

- `DD(val, grad, hess)` définit un nombre `DD` à partir de ses 3 données membres `val`, `grad` et `hess`. Il n'est pas utile de préciser `N` car la dimension est déduite de `length(grad)`.

- `DD(val, grad)` définit un nombre `DD` à partir de `val` et `grad` et fixe `hess` à la matrice nulle de dimension cohérente avec celle de `grad`.

- `DD{N}(val)` définit un nombre `DD` à partir de `val` uniquement. Il est alors nécessaire de préciser dans la construction la valeur de `N` qui ne peut se déduire de `val`. `grad` et `hess` sont alors initialisés à des valeurs nulles de dimension cohérente avec `N`.


Il est naturel de surcharger les opérateurs de base de la façon suivante

$$
\begin{align*}
\left(a+bᵀɛ+\frac{1}{2}ɛᵀcɛ\right)+\left(a'+b'ᵀɛ+\frac{1}{2}ɛᵀc'ɛ\right)&=(a+a')+(b+b')ᵀɛ+\frac{1}{2}ɛᵀ(c+c')ɛ\\
\left(a+bᵀɛ+\frac{1}{2}ɛᵀcɛ\right)-\left(a'+b'ᵀɛ+\frac{1}{2}ɛᵀc'ɛ\right)&=(a-a')+(b-b')ᵀɛ+\frac{1}{2}ɛᵀ(c-c')ɛ\\
\left(a+bᵀɛ+\frac{1}{2}ɛᵀcɛ\right)*\left(a'+b'ᵀɛ+\frac{1}{2}ɛᵀc'ɛ\right)&=(aa')+(ab'+a'b)ᵀɛ+\frac{1}{2}ɛᵀ(ac'+a'c+bb'ᵀ+b'bᵀ)ɛ
\end{align*}
$$

1. Compléter la surcharge des opérateurs `-` et `*` dans le code.  
   Remarque : la syntaxe `where {N}` rappelle que `N` est un paramètre qui sera remplacé à la compilation par la valeur voulue.  

1. On choisit de construire l'opérateur de division à partir de la définition préalable de la function d'inversion `inv`. Établir à la main l'expression des données membres `val` et `grad` et fixe `hess` de l'inverse d'un nombre de type `DD{N}` par identification en écrivant que la mutiplication d'un nombre par son inverse correspond à `val=1`, `grad=0` et `hess=0`. Compléter le code de la fonction `inv(x::DD{N}) where {N}`.

1. On se place en dimension $N=2$. On peut donc introduire les deux nombres

   ```julia
   εˣ = DD(0,[1,0])
   εʸ = DD(0,[0,1])
   ```

   Tester les développements des opérateurs sur des cas simples `1+εˣ`, `1+εˣ+εʸ` et sur des cas plus complexes `(1+εˣ)*(2+3εʸ)` et `εˣ/(1+εˣ+εʸ)` en comparant avec les valeurs calculées à la main. Par exemple

   $$
   ε^x ε^y=\frac{1}{2} \begin{pmatrix}ε^x \\ ε^y\end{pmatrix}^T \begin{pmatrix}0 & 1 \\ 1 & 0\end{pmatrix}\begin{pmatrix}ε^x \\ ε^y\end{pmatrix}
   ⟹
   \left(0, \begin{pmatrix}1 \\ 0\end{pmatrix}, \begin{pmatrix}0 & 0 \\ 0 & 0\end{pmatrix}\right)
   *
   \left(0, \begin{pmatrix}0 \\ 1\end{pmatrix}, \begin{pmatrix}0 & 0 \\ 0 & 0\end{pmatrix}\right)
   =
   \left(0, \begin{pmatrix}0 \\ 0\end{pmatrix}, \begin{pmatrix}0 & 1 \\ 1 & 0\end{pmatrix}\right)
   $$

1. Obtenir le gradient et la hessienne de $f(x,y)=\frac{(x+y)^2}{2}$ par différentiation automatique au point $(1,1)$.

1. Soit un entier arbitraire $n$, $a$, $b$ et $c$ respectivement un scalaire de $[0,1]$, un vecteur de $[0,1]^n$ et une matrice symétrique de $[0,1]^{n×n}$ aléatoires. Implémenter la fonction $f(x)=a+bᵀx+\frac{1}{2}xᵀcx$ pour $x∈\mathbb{R}^n$.  

   En exploitant judicieusement le vecteur de nombres

   ```julia
   ε = [DD(0,(1:n .== i)) for i in 1:n]
   ```

   et un tirage aléatoire d'un vecteur $x$, vérifier par différentiation automatique que $∇f(x)=b+cx$ et $∇²f(x)=c$.

1. On souhaite maintenant réaliser un programme permettant de minimiser une fonction $J$ arbitraire dépendant de plusieurs variables

   $$
   \min_{(p_1,\ldots,p_n)\in\mathbb{R}^n} J(p_1,\ldots,p_n)
   $$

   On suppose que le problème revient à chercher un vecteur de paramètres annulant son gradient par la méthode de Newton-Raphson. Autrement dit à chaque étape il est nécessaire d'évaluer le gradient ainsi que la hessienne de $J$
   
   $$
   p^{k+1}=p^{k}-∇²J(p^{k})^{-1}∇J(p^{k})
   $$
   
   Écrire une fonction générique `minimizer` prenant comme entrées un vecteur de paramètres initiaux `p⁰`, une fonction `J`, un nombre maximal d'itérations `maxiter` (par défaut `100`)  et un paramètre d'arrêt `ϵ` (par défaut `1.e-15`). On considérera un algorithme de Newton-Raphson avec un critère d'arrêt $\lVert ∇J(p^{k}\rVert<ϵ$.

1. Application n°1.

   On dispose de $n$ points $(x_i, y_i)$ d'une fonction inconnue $y = f(x)$.
   
   ```julia
   x = [0.0; 0.1; 0.2; 0.3; 0.4; 0.5; 0.6; 0.7; 0.8; 0.9; 1.0]
   y = [0.6761488864859304; 0.6345697680852508; 0.6396283580587062; 0.6132010027973919;
      0.5906142598705267; 0.5718728461471725; 0.5524549902830562; 0.538938885654085;
      0.5373495476994958; 0.514904589752926; 0.49243437874655027]
   ```
   
   On souhaite approximer $f$ par une fonction de la forme

   $$
   \widetilde f(x) = \frac{a}{b + x}
   $$
   en minimisant
   $$
   \sum_{i=1}^{n} |\widetilde f(x_i) - y_i|^2
   $$

   Écrire un code formant la fonction à minimiser et appliquer `minimizer` pour obtenir la meilleure approximation $\widetilde f$. Tracer sur le même graphe les points donnés et la fonction approximante.

1. Application n°2.

   On se donne $n$ nouveaux points $(x_i, y_i)$ d'une fonction inconnue $y = f(x)$
   
    ```julia
   x = [0.0; 0.1; 0.2; 0.3; 0.4; 0.5; 0.6; 0.7; 0.8; 0.9; 1.0]
   y = [-0.9187980789440975; -0.6159791344678258; -0.25568734869121856;
      -0.14269370171581808; 0.3094396057228459; 0.6318327173549161;
      0.8370437988106428; 1.0970402798788812; 1.6057799131867696;
      1.869090784869698; 2.075369730726694]
   ```  
   
   et on souhaite approximer $f$ par une fonction affine

   $$
   \widetilde f(x) = ax+b
   $$

   en minimisant la somme des distances euclidiennes entre les points et la droite définie par $\widetilde f$. Etant donné que la distance entre un point $(x_i,y_i)$ et la ligne droite est donnée par

   $$
   \frac{\lvert y_i - a x_i - b \rvert}{\sqrt{1+a^2}}
   $$

   la fonction objectif à minimiser s'écrit

   $$
   J(a, b) := \sum_{i=1}^{n} \frac{ \left( y_i - a x_i - b \right)^2 }{1+a^2}
   $$

   Trouver les paramètres optimaux $a$ et $b$ à l'aide de `minimizer` et tracer la droite ainsi que les points.

In [ ]:
import Base: +, -, *, /, inv, conj, convert, promote_rule
using LinearAlgebra

struct DD{N} <: Number
    val::Float64
    grad::Vector{Float64}
    hess::Symmetric{Float64, Matrix{Float64}}
    DD(val::Real, grad, hess) = new{length(grad)}(val, grad, Symmetric(hess))
    DD(val::Real, grad) = new{length(grad)}(val, grad, Symmetric(zeros(length(grad),length(grad))))
    DD{N}(val::Real) where {N} = new{N}(val, zeros(N), Symmetric(zeros(N,N)))
end
val(x::DD{N}) where {N} = x.val
grad(x::DD{N}) where {N} = x.grad
hess(x::DD{N}) where {N} = x.hess
conj(x::DD{N}) where {N} = DD(conj(x.val),conj(x.grad),conj(x.hess))
+(x::DD{N}, y::DD{N}) where {N} = DD(x.val+y.val,x.grad+y.grad,x.hess+y.hess)
-(x::DD{N}, y::DD{N}) where {N} = # à compléter ici
*(x::DD{N}, y::DD{N}) where {N} = # à compléter ici
inv(x::DD{N}) where {N} = # à compléter ici
/(x::DD{N}, y::DD{N}) where {N} = x*inv(y)
convert(::Type{DD{N}}, x::Real) where {N} = DD{N}(x)
promote_rule(::Type{DD{N}}, ::Type{<:Real}) where {N} = DD{N}
Base.show(io::IO,x::DD{N}) where {N} = print(io,x.val," + ",x.grad,"ᵀ𝛆"," + ½ 𝛆ᵀ",x.hess,"𝛆")

In [ ]:
εˣ = DD(0,[1,0])
εʸ = DD(0,[0,1])
@show 1+εˣ
# Votre code de vérification sur plusieurs expressions ici
;

In [ ]:
f(x,y) = (x+y)^2/2
@show # Votre code de calcul du gradient et de la hessienne de f en (1,1) ici

In [ ]:
function test_diff(n)
    # Votre code ici
end

test_diff(1)
test_diff(10)
test_diff(100)


In [ ]:
function minimizer(p⁰, J::Function, ϵ = 1.e-15, maxiter = 100)
    n = length(p⁰)
    p = p⁰ + [DD(0,(1:n .== i)) for i in 1:n]
    # Votre code ici
end

In [ ]:
x = [0.0; 0.1; 0.2; 0.3; 0.4; 0.5; 0.6; 0.7; 0.8; 0.9; 1.0]
y = [0.6761488864859304; 0.6345697680852508; 0.6396283580587062; 0.6132010027973919;
     0.5906142598705267; 0.5718728461471725; 0.5524549902830562; 0.538938885654085;
     0.5373495476994958; 0.514904589752926; 0.49243437874655027]
f(a,b) = x -> a / (b+x)
# Votre code ici

In [ ]:
x = [0.0; 0.1; 0.2; 0.3; 0.4; 0.5; 0.6; 0.7; 0.8; 0.9; 1.0]
y = [-0.9187980789440975; -0.6159791344678258; -0.25568734869121856;
     -0.14269370171581808; 0.3094396057228459; 0.6318327173549161;
     0.8370437988106428; 1.0970402798788812; 1.6057799131867696;
     1.869090784869698; 2.075369730726694]
f(a,b) = x -> a*x+b
# Votre code ici